### Lotame + LiveRamp + AIQ - File Creation


In [116]:
import pandas as pd
import warnings

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
warnings.filterwarnings('ignore')

### Audience Dataframe Creation

In [117]:
audiences = {
    'Country': ['AU', 'AU', 'AU', 'BR', 'BR', 'BR', 'CA', 'CA', 'CA', 'FR', 'FR'],
    'Number': [1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2],
    'Provider': ['Lotame', 'AIQ', 'LR', 'Lotame', 'AIQ', 'LR', 'Lotame', 'AIQ', 'LR', 'Lotame', 'AIQ'],
    'Audience ID': ['dgfrrpdw7ov', 'd2q11zclsgv', 'duxo79h5t8o', 'dkgud3s0umw', 'dwm57ismy2g', 'd3ok01g6usv',
             'dv7sm9mltvm', 'd7qd2wu3gkb', 'dw1v0i89ctr', 'd6m18ot8p0w', 'd14s5u5pfra']
}

df_audiences = pd.DataFrame(audiences)

df_audiences


,Country,Number,Provider,Audience ID
0,AU,1,Lotame,dgfrrpdw7ov
1,AU,2,AIQ,d2q11zclsgv
2,AU,3,LR,duxo79h5t8o
3,BR,1,Lotame,dkgud3s0umw
4,BR,2,AIQ,dwm57ismy2g
5,BR,3,LR,d3ok01g6usv
6,CA,1,Lotame,dv7sm9mltvm
7,CA,2,AIQ,d7qd2wu3gkb
8,CA,3,LR,dw1v0i89ctr
9,FR,1,Lotame,d6m18ot8p0w


In [118]:
audience_codes = '(dgfrrpdw7ov|d2q11zclsgv|duxo79h5t8o|dkgud3s0umw|dwm57ismy2g|d3ok01g6usv|dv7sm9mltvm|d7qd2wu3gkb|dw1v0i89ctr|d6m18ot8p0w|d14s5u5pfra)'


### GCM Data Import - Impressions & Clicks

In [119]:
df_impressions = pd.read_csv(r'C:\Users\Rafael_Fagundes\Downloads\1153793_Lotame__LiveRamp__AIQ_test_-_AU_BR_CA__FR_20230815_064016_4239885799.csv', skiprows=10,skipfooter=1, parse_dates=['Date'])

In [120]:
# Filter by Audience IDs
df_impressions_filtered = df_impressions[df_impressions['Placement'].str.contains(audience_codes, case=False, regex=True)]

# Convert to datetime for the merge
df_impressions_filtered['Date'] = pd.to_datetime(df_impressions_filtered['Date'])

# Remove any '-'
df_impressions_filtered.replace('-', 0, inplace=True)

# Change Reach and Frequency datatypes for the sum
df_impressions_filtered['Impressions'] = df_impressions_filtered['Impressions'].astype(float)
df_impressions_filtered['Clicks'] = df_impressions_filtered['Clicks'].astype(float)

# Group by Date and Placement and sum Reach and Frequency
df_impressions_group = df_impressions_filtered.groupby(['Date', 'Placement'])[['Impressions', 'Clicks']].sum().reset_index()

### GCM Data Import - Reach & Frequency


In [121]:
df_reach = pd.read_csv(r'C:\Users\Rafael_Fagundes\Downloads\1153793_Lotame__LiveRamp__AiQ_-_Reach_20230815_091154_4239979589.csv', skiprows=10,skipfooter=1, parse_dates=['Date'])

In [122]:
# Filter by Audience IDs
df_reach_filtered = df_reach[df_reach['Placement'].str.contains(audience_codes, case=False, regex=True)]

# Convert to datetime for the merge
df_reach_filtered['Date'] = pd.to_datetime(df_reach_filtered['Date'])

# Remove any '-'
df_reach_filtered.replace('-', 0, inplace=True)

# Rename columns: Reach and Frequency
df_reach_filtered.rename(columns={'Unique Reach: Impression Reach (Co-Viewed)': 'Reach', 'Unique Reach: Average Impression Frequency (Co-Viewed)': 'Frequency'}, inplace=True)

# Change Reach and Frequency datatypes for the sum
df_reach_filtered['Reach'] = df_reach_filtered['Reach'].astype(float)
df_reach_filtered['Frequency'] = df_reach_filtered['Frequency'].astype(float)

# Group by Date and Placement and sum Reach and Frequency
df_reach_group = df_reach_filtered.groupby(['Date', 'Placement'])[['Reach', 'Frequency']].sum().reset_index()


### Merge Audience Dataframe with GCM File

In [127]:
df_merge = pd.merge(df_impressions_group, df_reach_group, on=['Date','Placement'], how='left')

df_merge['Audience ID'] = df_merge['Placement'].str.lower().str.extract(audience_codes)

df_final = pd.merge(df_merge, df_audiences, on=['Audience ID'], how='left')

In [128]:
df_final

,Date,Placement,Impressions,Clicks,Reach,Frequency,Audience ID,Country,Number,Provider
0,2023-08-12,DV360_CRM-1PD CRM_STANDARD BANNER_160x600_DCPM_INTEL CCF_STD_FY24Q3_d14s5u5pfra-test-reactfullaiqv2-1pcrm_FR CSB_AIQ,20.0,0.0,0.0,0.00,d14s5u5pfra,FR,2,AIQ
1,2023-08-12,DV360_CRM-1PD CRM_STANDARD BANNER_300x250_DCPM_INTEL CCF_STD_FY24Q3_d14s5u5pfra-test-reactfullaiqv2-1pcrm_FR CSB_AIQ,643.0,0.0,173.0,3.72,d14s5u5pfra,FR,2,AIQ
2,2023-08-12,DV360_CRM-1PD CRM_STANDARD BANNER_300x600_DCPM_INTEL CCF_STD_FY24Q3_d14s5u5pfra-test-reactfullaiqv2-1pcrm_FR CSB_AIQ,117.0,0.0,0.0,0.00,d14s5u5pfra,FR,2,AIQ
3,2023-08-12,DV360_CRM-1PD CRM_STANDARD BANNER_728x90_DCPM_INTEL CCF_STD_FY24Q3_d14s5u5pfra-test-reactfullaiqv2-1pcrm_FR CSB_AIQ,108.0,0.0,0.0,0.00,d14s5u5pfra,FR,2,AIQ
4,2023-08-13,DV360_CRM-1PD CRM_STANDARD BANNER_160x600_DCPM_INTEL CCF_STD_FY24Q3_d14s5u5pfra-test-reactfullaiqv2-1pcrm_FR CSB_AIQ,54.0,0.0,0.0,0.00,d14s5u5pfra,FR,2,AIQ
5,2023-08-13,DV360_CRM-1PD CRM_STANDARD BANNER_300x250_DCPM_INTEL CCF_STD_FY24Q3_d14s5u5pfra-test-reactfullaiqv2-1pcrm_FR CSB_AIQ,1223.0,2.0,294.0,4.15,d14s5u5pfra,FR,2,AIQ
6,2023-08-13,DV360_CRM-1PD CRM_STANDARD BANNER_300x600_DCPM_INTEL CCF_STD_FY24Q3_d14s5u5pfra-test-reactfullaiqv2-1pcrm_FR CSB_AIQ,148.0,0.0,0.0,0.00,d14s5u5pfra,FR,2,AIQ
7,2023-08-13,DV360_CRM-1PD CRM_STANDARD BANNER_728x90_DCPM_INTEL CCF_STD_FY24Q3_d14s5u5pfra-test-reactfullaiqv2-1pcrm_FR CSB_AIQ,164.0,0.0,0.0,0.00,d14s5u5pfra,FR,2,AIQ
8,2023-08-14,DV360_CRM-1PD CRM_STANDARD BANNER_160x600_DCPM_INTEL CCF_STD_FY24Q3_d14s5u5pfra-test-reactfullaiqv2-1pcrm_FR CSB_AIQ,72.0,0.0,0.0,0.00,d14s5u5pfra,FR,2,AIQ
9,2023-08-14,DV360_CRM-1PD CRM_STANDARD BANNER_300x250_DCPM_INTEL CCF_STD_FY24Q3_d14s5u5pfra-test-reactfullaiqv2-1pcrm_FR CSB_AIQ,1043.0,1.0,287.0,3.63,d14s5u5pfra,FR,2,AIQ


### Save Final File

In [129]:
df_final.to_csv(r'csv\df.csv')